# Model Evaluation

In [ ]:
# Import libraries
import json
import os
from pneumonia_detector.model import PneumoniaClassifier
from pneumonia_detector.preprocess import XrayDataset
import torch
from torchvision import transforms
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

### Load in Classifier

In [ ]:
# Set model directory and model filename 
model_dir = ""
model_filename = ""

In [ ]:
# Load in parameter dictionary, then load in model and send to device
with open(os.path.join(model_dir, f"{model_filename}_params.json"), "r") as f:
        params = json.load(f)
    
device = params["device"]
model = PneumoniaClassifier(image_size=params["image_size"])
model.load_state_dict(torch.load(os.path.join(model_dir, model_filename), map_location=device))
model.to(device)

### Load in test data

In [ ]:
# Set test directory
test_dir = "../data/chest_xray/test/"

In [ ]:
# Set transformations to carry out on test images
test_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4823, 0.4823, 0.4823], std=[0.2363, 0.2363, 0.2363]),
])

In [ ]:
# Create test set Dataset object
test_dataset = XrayDataset(root_dir=test_dir, transform=test_transforms)

In [ ]:
len(test_dataset)

In [ ]:
# Run evalution on test set
model.eval()
with torch.no_grad():
    y_preds, y_true = zip(*[(model(item[0].unsqueeze(0)).argmax().item(), item[1]) for item in test_dataset])

### Evaluate Performance

In [ ]:
# Get accuracy
accuracy_score(y_true, y_preds)

In [ ]:
# Check confusion matrix
confusion_matrix(y_true, y_preds)

In [ ]:
# Classification report
print(classification_report(y_true, y_preds))

In [ ]:
# Plot confusion matrix
cm = confusion_matrix(y_true, y_preds)
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Normal", "Pneumonia"])
cm_display.plot()